In [ ]:
'''
pip install --no-cache-dir \
    ipywidgets==8.1.8 \
    lightning==2.6.0 \
    lightning-utilities==0.15.2 \
    nbformat==5.10.4 \
    neuralforecast==3.1.2 \
    numpy==2.2.6 \
    openpyxl==3.1.5 \
    optuna==4.6.0 \
    plotly==6.5.0 \
    pytorch-lightning==2.6.0 \
    scikit-learn==1.7.2 \
    statsforecast==2.0.3 \
    torch==2.9.1 \
    torchmetrics==1.8.2
'''

# Funções auxiliares

In [ ]:
from neuralforecast.losses.pytorch import HuberLoss
from sklearn.metrics import root_mean_squared_error
from torch.optim.lr_scheduler import StepLR
from neuralforecast import NeuralForecast
from IPython.display import clear_output
from torch.utils.data import DataLoader
from neuralforecast.models import LSTM
from typing import Dict, Any, Optional
from datetime import datetime
from torch.optim import AdamW
from pathlib import Path
import pandas as pd
import numpy as np
import random
import torch
import yaml
import sys
import os

global_seed = 42

In [ ]:
# Define as Seed Globais para Reprodutibilidade

# =============================================================================
# 1. FUNÇÕES AUXILIARES (IGUAIS AO ANTERIOR)
# =============================================================================
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def set_reproducibility(seed=42):
    print(f"🔒 Aplicando configurações de reprodutibilidade total (Seed: {seed})...")
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.set_float32_matmul_precision('medium') # 'high' usa TF32 que pode variar. 'medium' (default) ou 'highest' são mais estáveis.
    
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except AttributeError:
        pass

# =============================================================================
# 2. O PATCH CIRÚRGICO (INJEÇÃO DE MÉTODO)
# =============================================================================
# Aqui está a diferença: não criamos uma classe nova.
# Nós entramos dentro da classe DataLoader do PyTorch e trocamos o cérebro dela.

# Verifica se já não aplicamos o patch para evitar recursão infinita se rodar a célula 2x
if not hasattr(DataLoader, '_original_init'):
    print("🔧 Injetando código determinístico no DataLoader original...")
    
    # Salva o __init__ original numa gaveta segura
    DataLoader._original_init = DataLoader.__init__

    # Define a nova função que vai substituir o construtor
    def deterministic_init(self, *args, **kwargs):
        # 1. Injeção do Generator
        if 'generator' not in kwargs:
            g = torch.Generator()
            # Usa o seed atual para criar o generator
            g.manual_seed(torch.initial_seed()) 
            kwargs['generator'] = g
        
        # 2. Injeção do Worker Init
        if kwargs.get('num_workers', 0) > 0 and 'worker_init_fn' not in kwargs:
            kwargs['worker_init_fn'] = seed_worker
            
        # 3. Chama o __init__ original que salvamos
        # Isso garante que a NeuralForecast continue funcionando como esperado
        DataLoader._original_init(self, *args, **kwargs)

    # Substitui o método na classe original
    DataLoader.__init__ = deterministic_init
    print("✅ DataLoader 'hackeado' com sucesso (Método In-Place).")
else:
    print("ℹ️ Patch já estava aplicado. Nenhuma ação necessária.")

# =============================================================================
# 3. EXECUÇÃO
# =============================================================================
set_reproducibility(global_seed)

In [ ]:
# Classe para facilitar o uso de cores no terminal
class CoresTerminal:
    """Contém códigos ANSI para colorir o output no terminal."""
    VERMELHO = '\033[91m'
    VERDE = '\033[92m'
    FIM = '\033[0m'

In [ ]:
# Funçao para avaliar previsões
def evaluate_simple_forecast(
    model,
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    model_name: str = 'LSTM',  
    split: str ="",
    inteiro: bool = False,
    log_transformed: bool = True
) -> pd.DataFrame:
    """
    Executa avaliação de previsão considerando o nome do modelo dinâmico.
    Faz a previsão sem o uso de janela deslizande (rolling forecast).
    """
    print(f"\n{split} Iniciando a previsão com o modelo: {model_name}...")

    # --- 1. Preparação dos Dados ---
    history_df = train_df.copy()
    future_data_to_evaluate = test_df.copy()

    # Conversão de datas
    history_df['ds'] = pd.to_datetime(history_df['ds'])
    future_data_to_evaluate['ds'] = pd.to_datetime(future_data_to_evaluate['ds'])

    # Filtra histórico para evitar vazamento de dados
    cutoff = future_data_to_evaluate['ds'].min()
    history_df = history_df[history_df['ds'] < cutoff]
  
    # Prepara df futuro para predict (sem y)
    futr_df = future_data_to_evaluate.drop(columns=["y"], errors='ignore')

    # --- 2. Geração da Previsão ---
    forecasts_df = model.predict(
        df=history_df,
        futr_df=futr_df
    )
    
    # Verificação de segurança: A coluna do modelo existe?
    if model_name not in forecasts_df.columns:
        # Tenta fallback inteligente ou erro
        cols_disponiveis = [c for c in forecasts_df.columns if c not in ['unique_id', 'ds']]
        raise ValueError(f"A coluna '{model_name}' não foi encontrada na previsão. Colunas disponíveis: {cols_disponiveis}")

    print("Previsão concluída. Combinando com dados reais...")

    # --- 3. Pós-processamento e Avaliação ---
    evaluation_df = forecasts_df.merge(
        future_data_to_evaluate[["unique_id", "ds", "y"]],
        on=["unique_id", "ds"],
        how="inner"
    )

    # Tratamento de Log (Reversão)
    if log_transformed:
        evaluation_df['y'] = np.expm1(evaluation_df['y'])
        evaluation_df[model_name] = np.expm1(evaluation_df[model_name]) # Usa model_name

    # Arredondamento
    if inteiro:
        evaluation_df[model_name] = evaluation_df[model_name].round().astype(int)
        evaluation_df['y'] = evaluation_df['y'].round().astype(int)

    # Cálculo da Diferença Percentual (passando o nome da coluna)
    evaluation_df = calcular_diferenca_percentual(evaluation_df, col_pred=model_name)

    print("Avaliação finalizada.")
    return evaluation_df



def calcular_diferenca_percentual(df: pd.DataFrame, col_pred: str = 'LSTM', col_real: str = 'y') -> pd.DataFrame:
    """
    Calcula a diferença percentual absoluta entre previsão e valor real.
    Retorna NaN para casos de divisão por zero ou dados ausentes (período de treino).
    """
    df = df.copy()
    
    # 1. Garante que as colunas sejam numéricas
    # 'errors="coerce"' é o segredo: ele transforma "-" e textos em NaN automaticamente
    pred_numeric = pd.to_numeric(df[col_pred], errors='coerce')
    real_numeric = pd.to_numeric(df[col_real], errors='coerce')
    
    # 2. Cálculo Vetorizado: |(Pred - Real) / Real| * 100
    # O Pandas/Numpy lida com NaNs automaticamente (NaN em qualquer operação resulta em NaN)
    diff_pct = ((pred_numeric - real_numeric) / real_numeric).abs() * 100
    
    # 3. Tratamento de Divisão por Zero
    # O cálculo acima gera 'inf' (infinito) se o real for 0.
    # Aqui substituímos 'inf' por NaN para manter a consistência.
    df['diferença_%'] = diff_pct.replace([np.inf, -np.inf], np.nan)
    
    # 4. Arredondamento (opcional, para limpeza visual)
    df['diferença_%'] = df['diferença_%'].round(2)
    
    return df

In [ ]:
# Funão que testa o modelo salvo e salva resultados em CSV
def teste_modelo(
    local,              # Path onde o modelo treinado está salvo
    csv_dir,            # Path do arquivo CSV onde os resultados serão acumulados
    treino_id,          # ID único do experimento/treino
    dataset,            # DataFrame completo (Treino + Validação + Teste)
    train_ds,           # DataFrame apenas de Treino (usado para metadados)
    val_ds,             # DataFrame de Validação (pode ser None)
    test_ds,            # DataFrame de Teste
    comentario,         # String com observações do analista
    nome_dataset,       # Nome do dataset (ex: 'V43')
    hiperparametros,    # Dicionário com configs do modelo
    incluir_treino=False # Flag pesada: se True, faz previsão no passado (In-Sample)
):
    """
    Carrega um modelo NeuralForecast salvo, gera previsões para Teste, Validação 
    e (opcionalmente) Treino, calcula métricas de erro e salva tudo em um CSV histórico.
    """

    print(f"\n>>> Iniciando Teste do Modelo carregado de: {local}")
    
    # Carrega o modelo pré-treinado
    model = NeuralForecast.load(path=f"{local}")
    
    # Define colunas padrão para garantir consistência
    colunas_finais = ['treino_id', 'unique_id', 'ds', 'y', 'y_pred', 'diferença_%', 'flag', 'dataset', 'modelo', 'comentario', 'data_treino']

    # -------------------------------------------------------------------------
    # FUNÇÃO AUXILIAR INTERNA
    # -------------------------------------------------------------------------
    def processar_dataframe(df_raw, flag_name):
        """
        Padroniza o dataframe de previsões: renomeia colunas, calcula erro e ajusta datas.
        """
        if df_raw is None or df_raw.empty:
            return None
        
        df_proc = df_raw.copy()
        
        # Identifica dinamicamente a coluna de previsão (que não seja unique_id, ds ou y)
        # O NeuralForecast costuma nomear a coluna com o nome do modelo (ex: LSTM, NHITS)
        cols_reservadas = ['unique_id', 'ds', 'y', 'cutoff']
        candidatos_pred = [c for c in df_proc.columns if c not in cols_reservadas]
        
        # Se houver colunas de previsão, pega a primeira (assume modelo único aqui)
        col_pred_name = candidatos_pred[0] if candidatos_pred else 'LSTM'
        
        # Padroniza nomes
        df_proc = df_proc.rename(columns={col_pred_name: 'y_pred'})
        
        # Garante apenas as colunas essenciais
        df_proc = df_proc[['unique_id', 'ds', 'y', 'y_pred']].copy()
        
        # Calcula Erro da diferença percentual
        df_proc = calcular_diferenca_percentual(df_proc, col_pred='y_pred', col_real='y')
        
        # Metadados
        df_proc['flag'] = flag_name
        
        # Padronização de Data para String (ISO format) para salvar no CSV sem problemas
        df_proc['ds'] = (pd.to_datetime(df_proc['ds'], errors='coerce')
                         .fillna(pd.Timestamp.now())
                         .dt.strftime('%Y-%m-%dT%H:%M:%S'))
        
        return df_proc

    # -------------------------------------------------------------------------
    # 1. PREVISÕES NO CONJUNTO DE TESTE (Obrigatório)
    # -------------------------------------------------------------------------
    print("... Gerando previsões de Teste")
    preds_test = evaluate_simple_forecast(
        model=model,
        train_df=dataset,
        test_df=test_ds,
        split="Teste"
    )
    dados_teste = processar_dataframe(preds_test, 'teste')

    # -------------------------------------------------------------------------
    # 2. PREVISÕES NO CONJUNTO DE VALIDAÇÃO (Se val_ds for diferente de None)
    # -------------------------------------------------------------------------
    dados_validacao = None
    if val_ds is not None:
        print("... Gerando previsões de Validação")
        preds_val = evaluate_simple_forecast(
            model=model,
            train_df=dataset,
            test_df=val_ds,
            split="Validação"
        )
        dados_validacao = processar_dataframe(preds_val, 'validacao')

    # -------------------------------------------------------------------------
    # 3. PREVISÕES NO CONJUNTO DE TREINO (In-Sample)
    # -------------------------------------------------------------------------
    # Prepara o dataframe base de treino (revertendo log)
    dados_treino = dataset[['unique_id', 'ds', 'y']].copy()
    dados_treino['y'] = np.expm1(dados_treino['y']) # Reversão Log1p
    
    # Define data de corte para separar o que é treino
    if val_ds is not None:
        data_inicio_corte = val_ds['ds'].min() # Se tiver dados de validação o treino é anterior a ele.
    else:
        data_inicio_corte = dados_teste['ds'].min() # Se não houver validação, usa o periodo anterio ao teste.
        
    # Filtra apenas datas anteriores ao corte (validação ou teste)
    dados_treino = dados_treino[pd.to_datetime(dados_treino['ds']) < pd.to_datetime(data_inicio_corte)].copy()
    
    if incluir_treino:
        print("... Iniciando 'predict_insample' (Isso pode demorar!)")
        # Gera previsões dentro da amostra de treino
        insample_df = model.predict_insample(step_size=hiperparametros['h'])
        
        # Reversão da transformação logarítmica nas previsões e valores reais retornados
        insample_df['y'] = np.expm1(insample_df['y'])
        
        # Encontra colunas de previsão no insample (ex: LSTM, Autoformer)
        cols_pred_insample = [c for c in insample_df.columns if c not in ['unique_id', 'ds', 'cutoff', 'y']]
        col_model = cols_pred_insample[0] if cols_pred_insample else 'LSTM'
        insample_df[col_model] = np.expm1(insample_df[col_model])

        # Remove o período de "aquecimento" (context window) onde as previsões são ruins/inexistentes
        contexto = hiperparametros.get('input_size', 0)
        min_cutoff_valido = pd.to_datetime(insample_df['cutoff'].min()) + pd.DateOffset(years=contexto)
        insample_df = insample_df[insample_df['cutoff'] >= min_cutoff_valido].copy()

        # Merge para trazer a previsão para o dataframe de treino original
        insample_preds = insample_df[['unique_id', 'ds', col_model]].rename(columns={col_model: 'y_pred'})
        
        # Garante tipos de dados compatíveis para o merge
        dados_treino['ds'] = pd.to_datetime(dados_treino['ds'])
        insample_preds['ds'] = pd.to_datetime(insample_preds['ds'])
        
        dados_treino = dados_treino.merge(insample_preds, on=['unique_id', 'ds'], how='left')
        
        # Processa erro e formatação
        dados_treino = calcular_diferenca_percentual(dados_treino, col_pred='y_pred', col_real='y')
        dados_treino['flag'] = 'treino'
        dados_treino['ds'] = dados_treino['ds'].dt.strftime('%Y-%m-%dT%H:%M:%S')
        
    else:
        # Se não incluir treino, cria DF vazio estruturado para não quebrar o concat
        dados_treino = pd.DataFrame(columns=['unique_id', 'ds', 'y', 'y_pred', 'diferença_%', 'flag'])

    # -------------------------------------------------------------------------
    # 4. CONSOLIDAÇÃO E METADADOS
    # -------------------------------------------------------------------------
    print("... Consolidando dados")
    
    # Lista de dataframes válidos (ignora None)
    dfs_para_concatenar = [df for df in [dados_treino, dados_validacao, dados_teste] if df is not None and not df.empty]
    dados_completos = pd.concat(dfs_para_concatenar, ignore_index=True)

    # Adiciona Metadados Globais
    dados_completos['treino_id'] = treino_id
    dados_completos['dataset'] = nome_dataset
    dados_completos['modelo'] = "LSTM"
    dados_completos['data_treino'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

    # Gera o texto do comentário detalhado
    def get_period_text(df_periodo):
        if df_periodo is None or df_periodo.empty: return "sem dados"
        min_year = df_periodo['ds'].dt.year.min()
        max_year = df_periodo['ds'].dt.year.max()
        return f"{min_year}" if min_year == max_year else f"{min_year} a {max_year}"

    txt_val = get_period_text(val_ds)
    txt_test = get_period_text(test_ds)
    txt_train = get_period_text(train_ds)

    full_comment = (
        f"{local}\n"
        f"Modelo LSTM. Treino: {txt_train}. Validação: {txt_val}. Teste: {txt_test}.\n"
        f"Dataset: {nome_dataset}.\n"
        f"Obs: {comentario}\n\n"
        f"Hyperparams:\n"
        f"input_size: {hiperparametros.get('input_size')}"
        f"h: {hiperparametros.get('h')}"
        f"lr: {hiperparametros.get('learning_rate')}"
        f"batch: {hiperparametros.get('batch_size')}"
        f"encoder n layers: {hiperparametros.get('encoder_n_layers')}"
        f"decoder layers: {hiperparametros.get('decoder_layers')}"
    )
    dados_completos['comentario'] = full_comment

    # Ordenação final das colunas
    # Garante que todas as colunas existem, se não, cria com NaN
    for col in colunas_finais:
        if col not in dados_completos.columns:
            dados_completos[col] = np.nan
            
    dados_completos = dados_completos[colunas_finais]

    # -------------------------------------------------------------------------
    # 5. SALVAMENTO NO ARQUIVO CSV (APPEND)
    # -------------------------------------------------------------------------
    
    # Verifica se o diretório existe, se não, cria
    pasta = os.path.dirname(csv_dir)
    if pasta and not os.path.exists(pasta):
        os.makedirs(pasta)

    if not os.path.exists(csv_dir):
        print(f"Arquivo {csv_dir} não existe. Criando novo arquivo.")
        df_final = dados_completos
    else:
        print(f"Adicionando resultados ao arquivo existente: {csv_dir}")
        df_antigo = pd.read_csv(csv_dir)
        # Concatena antigo com novo
        df_final = pd.concat([df_antigo, dados_completos], ignore_index=True)

    # Salva
    df_final.to_csv(csv_dir, index=False)
    
    clear_output()
    print(f"✅ Sucesso! Previsões e métricas salvas em: {csv_dir}")

In [ ]:
# Função para ler hiperparâmetros de um arquivo YAML do PyTorch Lightning

def get_hyperparameters_from_yaml(yaml_path: str) -> Optional[Dict[str, Any]]:
    """
    Lê um arquivo hparams.yaml gerado pelo PyTorch Lightning e extrai hiperparâmetros.
    
    Args:
        yaml_path (str): Caminho para o arquivo .yaml.

    Returns:
        dict: Dicionário com os parâmetros. Retorna valores padrão (None) para chaves ausentes.
        None: Se o arquivo não existir ou estiver corrompido.
    """
    
    # 1. Verificação de existência do arquivo antes de tentar abrir
    if not os.path.exists(yaml_path):
        print(f"ERRO: Arquivo não encontrado: {yaml_path}")
        return None

    # 2. Leitura do YAML
    try:
        with open(yaml_path, 'r', encoding='utf-8') as f:
            # UnsafeLoader é necessário pois o PL salva tags de objetos Python (!!python/object...)
            config_data = yaml.load(f, Loader=yaml.UnsafeLoader)
            
        if not config_data:
            print(f"AVISO: O arquivo {yaml_path} está vazio.")
            return None
            
    except yaml.YAMLError as e:
        print(f"ERRO: YAML corrompido ou inválido: {e}")
        return None
    except Exception as e:
        print(f"ERRO: Falha inesperada ao ler {yaml_path}: {e}")
        return None

    # Tratamento especial para dicionários aninhados (optimizer_kwargs)
    opt_kwargs = config_data.get('optimizer_kwargs') or {} # Garante que seja dict se for None
    
    hyperparameters = {
        # Parâmetros Estruturais
        'encoder_n_layers': config_data.get('encoder_n_layers'),
        'encoder_hidden_size': config_data.get('encoder_hidden_size'),
        'decoder_layers': config_data.get('decoder_layers'),
        'decoder_hidden_size': config_data.get('decoder_hidden_size'),
        'input_size': config_data.get('input_size'),
        
        # Parâmetros de Treino
        'learning_rate': config_data.get('learning_rate'),
        'batch_size': config_data.get('batch_size'),
        'steps': config_data.get('max_steps'),
        
        # Mapeamentos com renomeação
        'dropout': config_data.get('encoder_dropout'), # Renomeia encoder_dropout -> dropout
        'weight_decay': opt_kwargs.get('weight_decay', 0.0), # Pega de dentro do kwargs ou retorna 0
        
        # IMPORTANTE: O script de teste usa 'h' (horizonte).
        # Tenta pegar 'h' ou 'horizon'. Se não achar, tenta inferir ou deixa None.
        'h': config_data.get('h', config_data.get('horizon')) 
    }

    return hyperparameters

In [ ]:
# Função para calcular a Média Simples do WMAPE por município

def calcular_media_wmape_simples(df: pd.DataFrame) -> float:
    """
    Calcula a Média Simples do WMAPE por município (unique_id).
    
    Esta função agrupa os dados por município, calcula o erro individual de cada um
    e, por fim, tira a média desses erros. Isso significa que todos os municípios
    têm o mesmo peso na métrica final, independentemente do volume de produção.
    
    Args:
        df (pd.DataFrame): DataFrame contendo as colunas 'unique_id', 'y' (real) e 'LSTM' (previsto).
        
    Returns:
        float: O valor médio do WMAPE entre todos os municípios.
    """
    
    # -------------------------------------------------------------------------
    # 1. Definição da lógica de cálculo (Função Interna)
    # -------------------------------------------------------------------------
    def _calcular_wmape_individual(grupo):
        """Calcula o WMAPE para um único grupo (município)."""
        
        # Converte para numpy array para garantir performance e operações vetoriais
        valores_reais = grupo['y'].values
        valores_previstos = grupo['LSTM'].values
        
        # Calcula o denominador: Soma absoluta dos valores reais
        soma_reais_abs = np.sum(np.abs(valores_reais))
        
        # PROTEÇÃO: Evita divisão por zero.
        # Se a soma dos reais for 0 (município sem produção), o erro é considerado 0.
        if soma_reais_abs == 0:
            return 0.0
            
        # Calcula o numerador: Soma absoluta das diferenças (erros)
        soma_erros_abs = np.sum(np.abs(valores_previstos - valores_reais))
        
        # Retorna o WMAPE deste município específico (Numerador / Denominador)
        return soma_erros_abs / soma_reais_abs

    # -------------------------------------------------------------------------
    # 2. Processamento Principal
    # -------------------------------------------------------------------------
    
    # Agrupa os dados pelo ID do município ('unique_id').
    # O método .apply() executa a função interna para CADA município separadamente.
    # O resultado (wmapes_por_municipio) será uma lista/Series com o erro de cada ID.
    wmapes_por_municipio = df.groupby('unique_id')[['y', 'LSTM']].apply(_calcular_wmape_individual)
    
    # Calcula a média simples de todos os WMAPEs individuais encontrados.
    # Ex: (WMAPE_Mun_A + WMAPE_Mun_B + ...) / Total_Municipios
    media_final = wmapes_por_municipio.mean()
    
    return media_final

In [ ]:
# Função para filtrar datasets por integridade dos dados (manter apenas municípios todos os registros no período de validação)
from typing import Tuple
def filtrar_datasets_por_integridade(
    train_df: pd.DataFrame, 
    val_df: pd.DataFrame, 
    test_df: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Filtra os datasets de treino, validação e teste para manter apenas os
    municípios ('unique_id') que possuem dados completos no período de validação.
    Ou seja, se validação tem 12 meses, mantém apenas os municípios que têm os 12 meses completos.

    Args:
        train_df (pd.DataFrame): DataFrame de treino.
        val_df (pd.DataFrame): DataFrame de validação. Usado como referência para a verificação.
        test_df (pd.DataFrame): DataFrame de teste.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: Uma tupla contendo os 
        DataFrames de treino, validação e teste devidamente filtrados.
    """
    print("--- Iniciando verificação de integridade dos dados ---")
    
    # Usa o DataFrame de validação como referência para a integridade.
    # 1. Conta o número de timestamps únicos que cada município DEVERIA ter.
    timestamps_esperados = len(val_df['ds'].unique())
    if timestamps_esperados == 0:
        print(f"{CoresTerminal.VERMELHO}ALERTA: O conjunto de validação está vazio. Nenhum filtro será aplicado.{CoresTerminal.FIM}")
        return train_df, val_df, test_df

    # 2. Conta quantos timestamps únicos cada município realmente POSSUI.
    contagem_por_id = val_df.groupby('unique_id')['ds'].nunique()
    
    # 3. Identifica os municípios com dados completos.
    ids_completos = contagem_por_id[contagem_por_id == timestamps_esperados].index
    
    # 4. Compara com o total de municípios para ver se a filtragem é necessária.
    ids_originais = val_df['unique_id'].nunique()
    
    if len(ids_completos) < ids_originais:
        total_removido = ids_originais - len(ids_completos)
        
        print(f"{CoresTerminal.VERMELHO}"
              "----------------------------------------------------------------------\n"
              "ALERTA: Inconsistência de dados encontrada.\n"
              f"Foram encontrados {total_removido} de {ids_originais} municípios com dados INCOMPLETOS no período de validação.\n"
              "Todos os datasets serão filtrados para manter apenas os municípios com dados completos.\n"
              f"----------------------------------------------------------------------{CoresTerminal.FIM}")
        
        # Filtra TODOS os dataframes para manter apenas os municípios com dados completos.
        # O uso de .copy() evita o SettingWithCopyWarning do pandas.
        train_filtrado = train_df[train_df['unique_id'].isin(ids_completos)].copy()
        val_filtrado = val_df[val_df['unique_id'].isin(ids_completos)].copy()
        test_filtrado = test_df[test_df['unique_id'].isin(ids_completos)].copy()
        
        # Se após a filtragem não sobrar nenhum dado, interrompe a execução.
        if val_filtrado.empty:
            print(f"{CoresTerminal.VERMELHO}ALERTA: Após a filtragem, não restaram dados válidos. Retornando DataFrames vazios.{CoresTerminal.FIM}")
            return train_filtrado, val_filtrado, test_filtrado
        
        print(f"Filtragem concluída. {len(ids_completos)} municípios mantidos.")
        return train_filtrado, val_filtrado, test_filtrado
        
    else:
        print(f"{CoresTerminal.VERDE}Verificação concluída. Todos os {ids_originais} municípios possuem dados completos no período de validação.{CoresTerminal.FIM}")
        return train_df, val_df, test_df

In [ ]:
# Função para carregar e processar o dataset

def get_dataset(dataset_file):
    """
    Carrega, processa e limpa o dataset, garantindo que todas as séries temporais
    para cada 'unique_id' estejam completas no intervalo de datas do dataset.
    """
    dataset = pd.read_csv(dataset_file)
    # --- Aplica transformação logarítmica ---
    log_cols = [
        'Área colhida (Hectares)',
        'target',
        'precomediocafe', # a partir de novembro 2025 (V37)
        'Área destinada à colheita (Hectares)'
    ]
    existing_log_cols = [col for col in log_cols if col in dataset.columns]
    if existing_log_cols:
        dataset[existing_log_cols] = dataset[existing_log_cols].apply(lambda x: np.log1p(x))

    # --- Renomeação e formatação para o padrão NeuralForecast ---
    dataset = dataset.rename(columns={
        "municipio": "unique_id",
        "ano": "ds",
        "target": "y"
    })
    # Ordenar por unique_id e ds (ano)
    dataset = dataset.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)
    dataset['ds'] = pd.to_datetime(dataset['ds'].astype(str) + '-12-31')
    return dataset

# Treinamento

In [ ]:
dataset_path = "Dataset/V43"

# sempre coloque a data no inicio do nome.
Local_treino =  "Teste_reprodutibilidade_2" # Nome do arquivo final com todas as predições
nome_dataset = "V43"
anos_validacao = [2023] #  Anos usados para validação (caso for validar com mais de um ano, coloque apenas o ano de inicio, pois o ano de teste sera o ano_val +h)

comentario = """teste de reprodutibilidade"""

In [ ]:
# Lista de parametros usados no modelo
hiperparametros = { 
    'h': 1,
    'input_size': -1, # Use -1 para janela de contexto máxima
    'batch_size': 32,
    'dropout': 0.3,
    'encoder_n_layers': 4,
    'learning_rate': 0.00013034723280377263,
    'encoder_hidden_size': 192,
    'decoder_layers': 1,
    'decoder_hidden_size': 64,
    'weight_decay': 0.0001,
    'steps': 1000
}

In [ ]:
for ano_val in anos_validacao:
    ano_teste = ano_val + hiperparametros['h'] # Define o ano de teste com base no horizonte. 
    dataset_path_ano = f"{dataset_path}/{ano_teste}"
    lista_datasets = sorted(os.listdir(dataset_path_ano)) # Garantir sempre a mesma ordem dos datasets
    for dataset_file in lista_datasets:
        
        treino_id = f"{dataset_file[:-4]}_{ano_teste}"
        dataset = get_dataset(f"{dataset_path_ano}/{dataset_file}")   

        # --- Coleta a lista variaveis exogenas ---
        exog_list = [col for col in dataset.columns.tolist() if col not in ["ds", "y", "unique_id"]]

        # --- Preparação do dataset de treino, validação e teste ---
        ano_teste = ano_val + hiperparametros['h'] # Define o ano de teste com base no horizonte. 
        train_ds = dataset[dataset['ds'].dt.year < ano_val].copy()
        val_ds = dataset[(dataset['ds'].dt.year >= ano_val) & (dataset['ds'].dt.year < ano_teste)].copy()
        test_ds = dataset[dataset['ds'].dt.year >= ano_teste].copy()

        # --- Aplica a função de filtragem para garantir a consistência ---
        clear_output()
        print(f"\n== Iniciando processamento para o dataset: {dataset_file} com validação em {ano_val} e teste em {ano_teste} ===\n")
        train_ds, val_ds, test_ds = filtrar_datasets_por_integridade(
            train_df=train_ds,
            val_df=val_ds,
            test_df=test_ds
        )
        print(f"Dados de Treino entre os anos de {train_ds['ds'].dt.year.min()} e {train_ds['ds'].dt.year.max()}: {len(train_ds)} registros")
        print(f"Dados de Validação entre os anos de {val_ds['ds'].dt.year.min() if val_ds is not None else 'N/A'} e {val_ds['ds'].dt.year.max() if val_ds is not None else 'N/A'}: {len(val_ds) if val_ds is not None else 0} registros")
        print(f"Dados de Teste entre os anos de {test_ds['ds'].dt.year.min()} e {test_ds['ds'].dt.year.max()}: {len(test_ds)} registros") 


        # -- Define o tamanho da janela de contexto (input_size) ---
        if hiperparametros['input_size'] == -1:
            hiperparametros['input_size'] = len(train_ds['ds'].unique()) - hiperparametros['h'] # Define o tamanho da janela de contexto como o total de anos de treino menos o horizonte

        print(f"\nTamanho da janela de contexto (input_size): {hiperparametros['input_size']}.\n")
        set_reproducibility(global_seed) # Reseta o estado global para garantir que este modelo começe do "zero absoluto", independente do modelo anterior.

        model = LSTM(
            h=hiperparametros['h'],
            input_size= hiperparametros['input_size'],
            batch_size=hiperparametros['batch_size'],
            scaler_type="revin",
            encoder_dropout=hiperparametros['dropout'],
            encoder_n_layers=hiperparametros['encoder_n_layers'],
            encoder_hidden_size=hiperparametros['encoder_hidden_size'],
            decoder_layers=hiperparametros['decoder_layers'],
            decoder_hidden_size=hiperparametros['decoder_hidden_size'],
            futr_exog_list=exog_list,
            learning_rate=hiperparametros['learning_rate'],
            max_steps=hiperparametros['steps'],
            loss=HuberLoss(delta=1.0),
            optimizer=AdamW,
            optimizer_kwargs={"weight_decay": hiperparametros['weight_decay']},
            lr_scheduler=StepLR,
            lr_scheduler_kwargs={"step_size": int(hiperparametros['steps'] * 0.5), "gamma": 0.1},
            random_seed=global_seed
        )

        nf = NeuralForecast(models=[model], freq="YE")
        nf.fit(df=train_ds)

        # --- Validando o modelo ---
        combined_df = evaluate_simple_forecast(
            model=nf,
            train_df=train_ds,
            test_df=val_ds
        )

        actual = combined_df["y"]
        predicted = combined_df["LSTM"]

        # Calcula as três versões da métrica
        score_media_simples = calcular_media_wmape_simples(combined_df)
        score_rmse = root_mean_squared_error(actual, predicted)
    
        # Imprime o score de validação
        print(f"\n\n --- Resultados da Validação ---")
        print(f"  - WMAPE: {score_media_simples:.4f}")
        print(f"  - RMSE:  {score_rmse:.4f}\n\n")

        # --- Salvando o modelo ---        
        local_save = f"./Treinos/{Local_treino}/Modelos/{treino_id}_({ano_teste})"
        csv_dir = f"./Treinos/{Local_treino}/{Local_treino}.csv"
        nf.save(
            path= local_save,
            overwrite=True)
        print(f"Modelo salvo em: {local_save}\n")
        
        # --- Avaliação ---
        teste_modelo(
            local = local_save,                   # Path onde o modelo treinado está salvo
            csv_dir = csv_dir,                    # Path do arquivo CSV onde os resultados serão acumulados
            treino_id = treino_id,                # ID único do experimento/treino
            dataset = dataset,                    # DataFrame completo (Treino + Validação + Teste)
            train_ds = train_ds,                  # DataFrame apenas de Treino (usado para metadados)
            val_ds = val_ds,                      # DataFrame de Validação (pode ser None)
            test_ds = test_ds,                    # DataFrame de Teste
            comentario = comentario,              # String com observações do analista
            nome_dataset = nome_dataset,          # Nome do dataset (ex: 'V43')
            hiperparametros = hiperparametros,    # Dicionário com configs do modelo
            incluir_treino=False                  # Se True, faz previsão no periodo de treino (In-Sample) (mais lento)
        )

In [ ]:
print("Resultado agregado dos clusters:\n")

# Constantes
SACA_KG = 60
PATH_REF = "Treinos/Modelo_Altitude_Ajustado/modelo_altitude_ajustado.csv"
PATH_AREA = "Dataset/Area_colhida_V2.csv"

df_area = pd.read_csv(PATH_AREA)

def processar_modelo(csv_path, df_area, colunas_para_converter):
    """
    Lê o CSV, filtra teste, faz merge com área, converte para sacas e agrupa por ano.
    """
    # Leitura e Pré-processamento básico
    df = pd.read_csv(csv_path)
    df = df[df['flag'] == 'teste'].copy()
    
    # Ajuste de Datas e Nomes
    df['Ano'] = pd.to_datetime(df['ds']).dt.year
    df.rename(columns={"unique_id": "Municipio"}, inplace=True)
    
    # Merge com Área
    df = df.merge(df_area, on=['Municipio', 'Ano'], how='left')
    
    # Cálculo Vetorizado (Sem loops)
    fator_conversao = (df['Area'] * 1000) / SACA_KG
    
    for col in colunas_para_converter:
        valores = pd.to_numeric(df[col], errors='coerce')
        df[col] = valores * fator_conversao
        
    # Agrupa por ano e soma
    return df.groupby('Ano')[colunas_para_converter].sum()

# 1. Processa Modelo Atual
resultados_atual = processar_modelo(csv_dir, df_area, ['y', 'y_pred'])

# 2. Processa Modelo de Referência
resultados_ref = processar_modelo(PATH_REF, df_area, ['y_pred'])
resultados_ref = resultados_ref.rename(columns={'y_pred': 'Referência'})

# 3. Junta os resultados
df_final = pd.concat([resultados_atual, resultados_ref], axis=1).round(3)

# Função de formatação no padrão brasileiro
def formatar_br(x):
    if pd.isna(x):
        return "—"
    int_part = int(x)
    dec_part = x - int_part
    int_str = f"{int_part:,}".replace(",", ".")
    dec_str = f"{abs(dec_part):.3f}".split('.')[1]
    return f"{int_str},{dec_str}"

# Aplica estilo com formatação personalizada
styled_df = (
    df_final
    .style
    .format(formatter=formatar_br, na_rep="—")
    .set_properties(**{'text-align': 'right'})
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'right')]},
        {'selector': '', 'props': [('border', '1px solid #ccc')]}
    ])
)

# Exibe no notebook
styled_df

# Inferencia pos treino


In [ ]:
def converter_colunas_para_minusculo(df):
    """
    Função que converte os nomes das colunas de um dataframe para minúsculas

    Parâmetros:
    - df: DataFrame que terá as colunas convertidas para minúsculas

    Retorna:
    - DataFrame com colunas em minúsculas
    """
    df_renomeado = df.rename(columns=lambda col: col.lower())
    return df_renomeado

In [ ]:
for i in range (0,200):
    local = f"Logs/trial_{i}/version_0"
    if not os.path.exists(local):
        os.makedirs(f"Logs/trial_{i}/version_0")

In [ ]:
df_final = pd.DataFrame()
ano = 2021
for cluster in range(5):   
    dataset = get_dataset(f"Dataset/V29 - Cluster 5/dataset_V29_cluster_{cluster}_metodo_1.csv") 
    model = NeuralForecast.load(path=f"Treinos/Antigos/Antigos_Antigos/Teste_Clusters/Modelos_antigos/IBGE - Cluster V5 Cluster {cluster} (2025)")

    #if ano == 2025:
    dataset = converter_colunas_para_minusculo(dataset)
        
    test_ds = dataset[dataset['ds'].dt.year == ano].copy()
    predictions = evaluate_simple_forecast(
        model=model,
        train_df=dataset,
        test_df=test_ds,
        split = "Teste"
    )

    dados_teste = predictions[['unique_id', 'ds', 'y', 'LSTM', 'diferença_%']].copy()
    dados_teste.columns = ['unique_id', 'ds', 'y', 'y_pred', 'diferença_%'] # Renomeando as colunas para o formato esperado
    dados_teste['flag'] = 'teste'
    dados_teste['cluster'] = cluster
    dados_teste['ds'] = pd.to_datetime(dados_teste['ds']).fillna(dados_teste['ds'])
    dados_teste['ds'] = (pd.to_datetime(dados_teste['ds'], errors='coerce').fillna(pd.Timestamp.now()).dt.strftime('%Y-%m-%dT%H:%M:%S'))
    df_final = pd.concat([df_final, dados_teste], ignore_index=True)

    
clear_output()

In [ ]:
df_area = pd.read_csv("Dataset/Area_colhida_V2.csv")
#df_area = pd.read_csv("Dataset/Area_colhida_projesao_2026.csv")
#df_area = pd.read_excel("Dataset/Projeções de áreas colhidas v2 - 2025 e 2026.xlsx")

df_area.rename(columns={"Área colhida (Hectares)": "Area"}, inplace=True)
for unique_id in df_final['unique_id'].unique():
    # Filtrar as linhas correspondentes no df_soma
    mask_final = df_final['unique_id'] == unique_id

    # Buscar área correspondente
    area_row = df_area[(df_area['Municipio'] == unique_id) & (df_area['Ano'] == ano)]
    
    if area_row.empty:
        print(f"Aviso: Área não encontrada para unique_id = {unique_id}")
        area_total = np.nan  # ou continue, ou defina um valor padrão
    else:
        area_total = area_row['Area'].iloc[0]

    # Aplicar a transformação SOMENTE às linhas desse unique_id
    if pd.notna(area_total):
        df_final.loc[mask_final, 'y_pred'] = (
            (df_final.loc[mask_final, 'y_pred'] * area_total) * 1000
        ) / 60
    else:
        df_final.loc[mask_final, 'y_pred'] = np.nan  # ou 0, dependendo da lógica

print(f"Ano: {ano} -> {df_final['y_pred'].sum()}")

In [ ]:
# Lista de parametros usados no modelo
hiperparametros = { 
    'h': 1,
    'input_size': 3, # Use -1 para janela de contexto máxima
    'batch_size': 32,
    'dropout': 0.3,
    'encoder_n_layers': 64,
    'learning_rate': 0.0001092083381402252,
    'encoder_hidden_size': 128,
    'decoder_layers': 2,
    'decoder_hidden_size': 128,
    'weight_decay': 0.0001,
    'steps': 200
}

In [ ]:
#cluster = 0
ano = 2021
#for ano in [2020, 2021, 2022, 2023, 2024]:
for cluster in range(5):
    dataset = get_dataset(f"Dataset/V29 - Cluster 5/dataset_V29_cluster_{cluster}_metodo_1.csv") 
    #dataset = converter_colunas_para_minusculo(dataset)

    local_modelo = f"Treinos/Antigos/Antigos_Antigos/Teste_Clusters/Modelos_antigos/IBGE - Cluster V5 Metodo 1 Modelo unico (2021)"
    csv_dir = "teste/IBGE_Cluster_V5_metodo_1_2021.csv"
    train_ds = dataset[dataset['ds'].dt.year < ano-1].copy()
    val_ds = dataset[dataset['ds'].dt.year == ano-1].copy()
    test_ds = dataset[dataset['ds'].dt.year == ano].copy()
    comentario = "dataset V29 com com as features escritas em minusculo. para padronizar de agora em diante."
    nome_dataset = "V29"
    treino_id = f"IBGE - Cluster V5 Metodo 1 Cluster {cluster} ({ano})"
    teste_modelo(local_modelo, csv_dir, dataset, train_ds, val_ds, test_ds, comentario, nome_dataset, h=hiperparametros['h'])

In [ ]:
df = pd.read_csv("teste/IBGE_Cluster_V5_metodo_1_2025.csv")
df_original = pd.read_csv("Treinoss/cluster_V5_Cluster_separados/cluster_V5_Cluster_separados.csv")

In [ ]:
df_novo = pd.concat([df_original, df], ignore_index=True)

In [ ]:
df_novo["treino_id"].unique()
df_novo.to_csv("Treinoss/cluster_V5_Cluster_separados/cluster_V5_Cluster_separados_2024.csv", index=False)